In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from collections import Counter
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.metrics import pairwise
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.stats import entropy
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from kmodes.kprototypes import KPrototypes
import pandas as pd
import re

In [ ]:
!pip install kmodes

# Processing the disease data


## Drug and disease- dataset 1

This is from kaggle Drug prescription to disease dataset

In [ ]:
# Read the CSV file into a DataFrame
drugdf1 = pd.read_csv("final.csv",index_col=None)
drugdf1 = drugdf1.drop_duplicates(subset=['disease', 'drug'])
drugdf1.head(1)

,Unnamed: 0,disease,drug
0,0,Alkylating Agent Cystitis,sodium bicarbonate


## Dataset 2

this is from drugs.com

In [ ]:
# Read the CSV file into a DataFrame
drugdf2 = pd.read_csv("drugs.csv")
drugdf2 = drugdf2.drop_duplicates(subset=['drug_name', 'medical_condition'])
# Display the first few rows of the DataFrame to verify it was read correctly
print(drugdf2.head(1))

     drug_name medical_condition  \
0  doxycycline              Acne   

                       medical_condition_description activity rx_otc  \
0  Acne Other names: Acne Vulgaris; Blackheads; B...      87%     Rx   

  pregnancy_category csa alcohol  rating  no_of_reviews  \
0                  D   N       X     6.8          760.0   

                       medical_condition_url  \
0  https://www.drugs.com/condition/acne.html   

                                drug_link  
0  https://www.drugs.com/doxycycline.html  


In [ ]:
drugdf2.rename(columns={'medical_condition': 'disease', 'drug_name': 'drug'}, inplace=True)

# Concatenate the two DataFrames
combined_df = pd.concat([drugdf1[['disease', 'drug']], drugdf2[['disease', 'drug']]])

# Convert values in 'disease' and 'drug' columns to lowercase
combined_df['disease'] = combined_df['disease'].str.lower()
combined_df['drug'] = combined_df['drug'].str.lower()

# Remove duplicates based on 'disease' and 'drug' columns
combined_df = combined_df.drop_duplicates(subset=['disease', 'drug'])
combined_df = combined_df.drop_duplicates(subset=['drug'])
combined_df = combined_df.reset_index(drop=True)

# Display the combined DataFrame
combined_df

,disease,drug
0,alkylating agent cystitis,sodium bicarbonate
1,alkylating agent cystitis,citric acid / sodium citrate
2,abdominal distension,bethanechol
3,abdominal distension,pamabrom
4,abdominal distension prior to abdominal x-ray,vasopressin
...,...,...
3961,weight loss,benzphetamine
3962,weight loss,fenfluramine
3963,weight loss,fendique er
3964,weight loss,imcivree


In [ ]:
from google.colab import files

combined_df.to_csv('drug_df.csv', index=False)
files.download('drug_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Read the CSV file into a DataFrame
drugdf2 = pd.read_csv("df_selected.csv")
combined_df = pd.read_csv("drug_df.csv")

In [ ]:
# Impute null values in 'Tot_Benes' column with 6
drugdf2 = drugdf2.fillna({'Tot_Benes': 6})

In [ ]:
# Define the states-to-region mapping dictionary
states = {
    'AK': 'West', 'AL': 'Southeast', 'AR': 'South',
    'AZ': 'Southwest', 'CA': 'West', 'CO': 'West',
    'CT': 'Northeast', 'DC': 'Southeast', 'DE': 'Northeast',
    'FL': 'Southeast', 'GA': 'Southeast', 'HI': 'West',
    'IA': 'Midwest', 'ID': 'West', 'IL': 'Midwest',
    'IN': 'Midwest', 'KS': 'Midwest', 'KY': 'Southeast',
    'LA': 'South', 'MA': 'Northeast', 'MD': 'Northeast',
    'ME': 'Northeast', 'MI': 'Midwest', 'MN': 'Midwest',
    'MO': 'Midwest', 'MS': 'Southeast', 'MT': 'West',
    'NC': 'Southeast', 'ND': 'Midwest', 'NE': 'Midwest',
    'NH': 'Northeast', 'NJ': 'Northeast', 'NM': 'Southwest',
    'NV': 'West', 'NY': 'Northeast', 'OH': 'Midwest',
    'OK': 'Southwest', 'OR': 'West', 'PA': 'Northeast',
    'RI': 'Northeast', 'SC': 'Southeast', 'SD': 'Midwest',
    'TN': 'Southeast', 'TX': 'Southwest', 'UT': 'West',
    'VA': 'Southeast', 'VT': 'Northeast', 'WA': 'West',
    'WI': 'Midwest', 'WV': 'Southeast', 'WY': 'West', 'PR': 'East'
}

# Map states to regions
drugdf2['Region'] = drugdf2['Prscrbr_State_Abrvtn'].map(states)


In [ ]:
# Convert 'Gnrc_Name' column in cluster_model1_df to lowercase
drugdf2['Gnrc_Name'] = drugdf2['Gnrc_Name'].str.lower()

In [ ]:
# Merge cluster_model1_df with combined_df on the 'Gnrc_Name' and 'drug' columns
merged_df = pd.merge(drugdf2, combined_df, how='left', left_on='Gnrc_Name', right_on='drug')

# Display the merged DataFrame
print(merged_df)

      Prscrbr_NPI Prscrbr_Last_Org_Name Prscrbr_First_Name  Prscrbr_City  \
0      1003007980                Bernal               Juan    Birmingham   
1      1003073735                  Fair               Keri       Chicago   
2      1003075094           Di Pascuale              Mario       El Paso   
3      1003078007                Polsak           Micholee   Kansas City   
4      1003009119                Mohler               Troy  Purcellville   
...           ...                   ...                ...           ...   
9995   1003055088               Barnett           Taneisha      Gastonia   
9996   1003066846                Younis             Naveed     Jonesboro   
9997   1003076894                Myrmoe              Jason         Fargo   
9998   1003007345                Harlow            Michael   Minneapolis   
9999   1003090390                 Skeif              Basel        Tucson   

     Prscrbr_State_Abrvtn  Prscrbr_State_FIPS               Prscrbr_Type  \
0          

## match again with the first word on the drug

In [ ]:
# Step 2: Check the 'disease' field which is null
null_disease_records = merged_df[merged_df['disease'].isnull()]
null_disease_records

,Prscrbr_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_City,Prscrbr_State_Abrvtn,Prscrbr_State_FIPS,Prscrbr_Type,Prscrbr_Type_Src,Brnd_Name,Gnrc_Name,...,GE65_Sprsn_Flag,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Bene_Sprsn_Flag,GE65_Tot_Benes,Region,disease,drug
3,1003078007,Polsak,Micholee,Kansas City,KS,20,Internal Medicine,S,Breo Ellipta,fluticasone/vilanterol,...,NaN,21.0,23.0,9058.23,690.0,*,NaN,Midwest,NaN,NaN
4,1003009119,Mohler,Troy,Purcellville,VA,51,Family Practice,S,Enalapril Maleate,enalapril maleate,...,NaN,13.0,23.0,292.68,690.0,*,NaN,Southeast,NaN,NaN
6,1003013616,Woschkolup,Kathleen,Greenville,SC,45,Neurology,S,Rytary,carbidopa/levodopa,...,NaN,211.0,339.4,352790.26,9979.0,#,NaN,Southeast,NaN,NaN
7,1003021734,Fisher,Gerald,Johnstown,PA,42,Dentist,S,Clindamycin Hcl,clindamycin hcl,...,#,NaN,NaN,NaN,NaN,#,NaN,Northeast,NaN,NaN
9,1003061532,Jacobs,Jesse,Santa Barbara,CA,6,Physician Assistant,S,Oxycodone-Acetaminophen,oxycodone hcl/acetaminophen,...,*,NaN,NaN,NaN,NaN,*,NaN,West,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,1003075516,Khan,Aftab,Hershey,PA,42,Psychiatry,S,Pramipexole Dihydrochloride,pramipexole di-hcl,...,NaN,0.0,0.0,0.00,0.0,#,NaN,Northeast,NaN,NaN
9992,1003055302,Alqwasmi,Abdel Hai,West Bend,WI,55,Hematology-Oncology,S,Breo Ellipta,fluticasone/vilanterol,...,NaN,12.0,12.0,4783.76,360.0,*,NaN,Midwest,NaN,NaN
9994,1003078254,Rutkowski,Paul,Wilkes Barre,PA,42,Internal Medicine,S,Metformin Hcl Er,metformin hcl,...,#,NaN,NaN,NaN,NaN,*,NaN,Northeast,NaN,NaN
9998,1003007345,Harlow,Michael,Minneapolis,MN,27,Psychiatry,S,Haloperidol Lactate,haloperidol lactate,...,NaN,0.0,0.0,0.00,0.0,#,NaN,Midwest,NaN,NaN


In [ ]:
# Step 3: For the records which have null in the 'disease' field, try to match the first word of 'Gnrc_Name' with 'drug' in combined_df
for index, row in null_disease_records.iterrows():
    first_words = re.split(r'\s|/|,', row['Gnrc_Name'].lower())
    for first_word in first_words:
        matching_drug = combined_df[combined_df['drug'].str.startswith(first_word)]
        if not matching_drug.empty:
            # Update 'disease' field in merged_df with the corresponding value from matching_drug
            merged_df.at[index, 'disease'] = matching_drug.iloc[0]['disease']
            break  # Exit the loop once a matching drug is found

# Display the merged DataFrame
print(merged_df)


      Prscrbr_NPI Prscrbr_Last_Org_Name Prscrbr_First_Name  Prscrbr_City  \
0      1003007980                Bernal               Juan    Birmingham   
1      1003073735                  Fair               Keri       Chicago   
2      1003075094           Di Pascuale              Mario       El Paso   
3      1003078007                Polsak           Micholee   Kansas City   
4      1003009119                Mohler               Troy  Purcellville   
...           ...                   ...                ...           ...   
9995   1003055088               Barnett           Taneisha      Gastonia   
9996   1003066846                Younis             Naveed     Jonesboro   
9997   1003076894                Myrmoe              Jason         Fargo   
9998   1003007345                Harlow            Michael   Minneapolis   
9999   1003090390                 Skeif              Basel        Tucson   

     Prscrbr_State_Abrvtn  ...  GE65_Bene_Sprsn_Flag GE65_Tot_Benes  \
0               

In [ ]:
filtered_df = merged_df[merged_df['disease'].isnull()]
filtered_df

,Prscrbr_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_City,Prscrbr_State_Abrvtn,Prscrbr_State_FIPS,Prscrbr_Type,Prscrbr_Type_Src,Brnd_Name,Gnrc_Name,...,GE65_Sprsn_Flag,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Bene_Sprsn_Flag,GE65_Tot_Benes,Region,disease,drug
55,1003084302,Bhangoo,Rupinder,Fishkill,NY,36,Obstetrics & Gynecology,S,Kariva,desog-e.estradiol/e.estradiol,...,NaN,0.0,0.0,0.00,0.0,#,NaN,Northeast,NaN,NaN
133,1003020645,Ilahi,Marium,Omaha,NE,31,Endocrinology,S,Single Use Swab,alcohol antiseptic pads,...,*,NaN,NaN,NaN,NaN,*,NaN,Midwest,NaN,NaN
187,1003065509,Thomas,Nicole,Jacksonville Beach,FL,12,Nurse Practitioner,S,Blisovi Fe,norethindrone-e.estradiol-iron,...,NaN,0.0,0.0,0.00,0.0,#,NaN,Southeast,NaN,NaN
311,1003004540,Cabanellas,Jennine,Wellington,FL,12,Ophthalmology,S,Rhopressa,netarsudil mesylate,...,#,NaN,NaN,NaN,NaN,#,NaN,Southeast,NaN,NaN
325,1003088915,Voigt,Amy,Metairie,LA,22,Neurology,S,Emgality Pen,galcanezumab-gnlm,...,NaN,0.0,0.0,0.00,0.0,#,NaN,South,NaN,NaN
443,1003077991,Hong,Alice,New York,NY,36,Ophthalmology,S,Rhopressa,netarsudil mesylate,...,NaN,30.0,37.7,10487.78,1081.0,*,NaN,Northeast,NaN,NaN
467,1003059981,Ahmed,Lotus,Brooklyn,NY,36,Internal Medicine,S,Alcohol Pads,alcohol antiseptic pads,...,NaN,58.0,99.3,3539.29,2950.0,#,NaN,Northeast,NaN,NaN
485,1003034398,Chiu,Misty,Yonkers,NY,36,Nurse Practitioner,S,Alcohol Pads,alcohol antiseptic pads,...,NaN,0.0,0.0,0.00,0.0,#,NaN,Northeast,NaN,NaN
496,1003063264,Siddiqui,Juwaria,Lincolnwood,IL,17,Family Practice,S,Alcohol Pads,alcohol antiseptic pads,...,NaN,17.0,20.4,800.90,612.0,*,NaN,Midwest,NaN,NaN
826,1003034851,Gust,Amy,Sevierville,TN,47,Family Practice,S,Single Use Swab,alcohol antiseptic pads,...,#,NaN,NaN,NaN,NaN,*,NaN,Southeast,NaN,NaN


In [ ]:
filtered_df['Gnrc_Name'].unique()

array(['desog-e.estradiol/e.estradiol', 'alcohol antiseptic pads',
       'norethindrone-e.estradiol-iron', 'netarsudil mesylate',
       'galcanezumab-gnlm', 'lipase/protease/amylase',
       'fremanezumab-vfrm', 'zafirlukast', 'istradefylline',
       'erenumab-aooe', 'ivabradine hcl', 'opicapone', 'vibegron',
       'apalutamide', 'alpha-1-proteinase inhibitor'], dtype=object)

In [ ]:
filtered_df['Brnd_Name'].nunique()

21

In [ ]:
# Drop rows with missing values in the 'disease' column
merged_df.dropna(subset=['disease'], inplace=True)


In [ ]:
from google.colab import files

merged_df.to_csv('drug_disease.csv', index=False)
files.download('drug_disease.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# model building

## Read the data that has both drug and disease

In [ ]:
df_disease = pd.read_csv('drug_disease.csv')

In [ ]:
df_disease.columns

Index(['Prscrbr_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name',
       'Prscrbr_City', 'Prscrbr_State_Abrvtn', 'Prscrbr_State_FIPS',
       'Prscrbr_Type', 'Prscrbr_Type_Src', 'Brnd_Name', 'Gnrc_Name',
       'Tot_Clms', 'Tot_30day_Fills', 'Tot_Day_Suply', 'Tot_Drug_Cst',
       'Tot_Benes', 'GE65_Sprsn_Flag', 'GE65_Tot_Clms', 'GE65_Tot_30day_Fills',
       'GE65_Tot_Drug_Cst', 'GE65_Tot_Day_Suply', 'GE65_Bene_Sprsn_Flag',
       'GE65_Tot_Benes', 'Region', 'disease', 'drug'],
      dtype='object')

In [ ]:
 del df_disease['drug']

In [ ]:
df_disease['disease'].nunique()

250

In [ ]:
df_disease.dropna(subset=['Region'],inplace = True)

In [ ]:
df_disease.shape

(9903, 24)

#Model 1 - K prototype

In [ ]:
model1_df = df_disease

In [ ]:
cluster_model1_df = df_disease[['Gnrc_Name','Tot_Clms', 'Tot_Drug_Cst', 'Tot_Day_Suply',
                                'Tot_Benes','Tot_30day_Fills','Region', 'disease']]


In [ ]:
# Define the columns to apply log transformation
columns_to_transform = ['Tot_Clms', 'Tot_Drug_Cst', 'Tot_Day_Suply', 'Tot_Benes', 'Tot_30day_Fills']

# Apply log transformation to the selected columns
for column in columns_to_transform:
    cluster_model1_df[column] = np.log(cluster_model1_df[column] + 1)  # Adding 1 to handle zero values

cluster_model1_df.head()

,Gnrc_Name,Tot_Clms,Tot_Drug_Cst,Tot_Day_Suply,Tot_Benes,Tot_30day_Fills,Region,disease
0,potassium chloride,4.700480,8.968666,8.694670,3.496508,5.298317,Southeast,hypokalemia
1,fluconazole,2.708050,4.112184,3.295837,1.945910,2.708050,Midwest,blastomycosis
2,acyclovir,4.043051,7.444214,7.916078,2.995732,4.564348,Southwest,varicella-zoster
3,fluticasone/vilanterol,3.091042,9.111539,6.538140,1.945910,3.178054,Midwest,atopic dermatitis
4,enalapril maleate,2.639057,5.682491,6.538140,1.945910,3.178054,Southeast,polycystic kidney disease


In [ ]:
# Convert dataframe to matrix
df_matrix = cluster_model1_df.to_numpy()
df_matrix

array([['potassium chloride', 4.700480365792417, 8.968666184042538, ...,
        5.298317366548036, 'Southeast', 'hypokalemia'],
       ['fluconazole', 2.70805020110221, 4.1121844803504315, ...,
        2.70805020110221, 'Midwest', 'blastomycosis'],
       ['acyclovir', 4.04305126783455, 7.444213561161819, ...,
        4.564348191467836, 'Southwest', 'varicella-zoster'],
       ...,
       ['famotidine', 3.4965075614664802, 5.890593936723008, ...,
        3.713572066704308, 'Midwest', 'gerd'],
       ['haloperidol lactate', 2.6390573296152584, 6.221683012094154,
        ..., 2.6390573296152584, 'Midwest', 'aggressive behavior'],
       ['nebivolol hcl', 3.6375861597263857, 9.524333989741718, ...,
        4.477336814478207, 'Southwest', 'mitral valve prolapse']],
      dtype=object)

##k-prototype

In [ ]:
# Define the indices of the numerical and categorical features
cat_cols = [0,6,7]

# Perform k-prototype clustering
kp = KPrototypes(n_clusters=4, init='Cao', n_init=5, verbose=1)
kp.fit(df_matrix, categorical=cat_cols)


Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 2881, ncost: 36995.68414424174
Run: 1, iteration: 2/100, moves: 890, ncost: 35937.758017527885
Run: 1, iteration: 3/100, moves: 308, ncost: 35832.856020122264
Run: 1, iteration: 4/100, moves: 158, ncost: 35814.92820345953
Run: 1, iteration: 5/100, moves: 76, ncost: 35809.42532678524
Run: 1, iteration: 6/100, moves: 82, ncost: 35803.146543095405
Run: 1, iteration: 7/100, moves: 58, ncost: 35800.16868026397
Run: 1, iteration: 8/100, moves: 47, ncost: 35798.069280128926
Run: 1, iteration: 9/100, moves: 40, ncost: 35796.725063078724
Run: 1, iteration: 10/100, moves: 45, ncost: 35795.1861847247
Run: 1, iteration: 11/100, moves: 48, ncost: 35793.770700284585
Run: 1, iteration: 12/100, moves: 24, ncost: 35793.31763478549
Run: 1, iteration: 13/100, moves: 18, ncost: 35793.10328733279
Run: 1, iteration: 14/10

KPrototypes(gamma=0.5630495217451406, n_clusters=4, n_init=5, verbose=1)

In [ ]:
#new column for cluster labels associated with each subject
model1_df['labels'] = kp.labels_

In [ ]:
model1_df.head()

,Prscrbr_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_City,Prscrbr_State_Abrvtn,Prscrbr_State_FIPS,Prscrbr_Type,Prscrbr_Type_Src,Brnd_Name,Gnrc_Name,...,GE65_Sprsn_Flag,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Bene_Sprsn_Flag,GE65_Tot_Benes,Region,disease,labels
0,1003007980,Bernal,Juan,Birmingham,AL,1,Interventional Cardiology,S,Potassium Chloride,potassium chloride,...,NaN,75.0,143.0,7086.79,4290.0,#,NaN,Southeast,hypokalemia,1
1,1003073735,Fair,Keri,Chicago,IL,17,Family Practice,S,Fluconazole,fluconazole,...,#,NaN,NaN,NaN,NaN,*,NaN,Midwest,blastomycosis,0
2,1003075094,Di Pascuale,Mario,El Paso,TX,48,Ophthalmology,S,Acyclovir,acyclovir,...,NaN,56.0,95.0,1708.94,2740.0,NaN,19.0,Southwest,varicella-zoster,3
3,1003078007,Polsak,Micholee,Kansas City,KS,20,Internal Medicine,S,Breo Ellipta,fluticasone/vilanterol,...,NaN,21.0,23.0,9058.23,690.0,*,NaN,Midwest,atopic dermatitis,2
4,1003009119,Mohler,Troy,Purcellville,VA,51,Family Practice,S,Enalapril Maleate,enalapril maleate,...,NaN,13.0,23.0,292.68,690.0,*,NaN,Southeast,polycystic kidney disease,0


##metrics

In [ ]:
for i in range(0,4):
   print ("Cluster",i,":", model1_df.groupby('labels').count().iloc[i]['Gnrc_Name'])

Cluster 0 : 1591
Cluster 1 : 3582
Cluster 2 : 1581
Cluster 3 : 3149


###entropy

In [ ]:
# Group by 'cluster' and count the occurrences of each drug in each cluster
cluster_counts = model1_df.groupby('labels')['Gnrc_Name'].value_counts().unstack(fill_value=0)

# Calculate the total count of drugs in each cluster
total_drugs_in_cluster = cluster_counts.sum(axis=1)

# Calculate the probability of each drug in each cluster
cluster_probabilities = cluster_counts.div(total_drugs_in_cluster, axis=0)

# Transpose the DataFrame to have drugs as rows and clusters as columns
cluster_probabilities_transposed = cluster_probabilities.T

# Calculate the entropy for each cluster
cluster_entropies = -(cluster_probabilities_transposed * np.log2(cluster_probabilities_transposed)).fillna(0)

# Sum the entropies for each cluster
cluster_entropies_sum = cluster_entropies.sum(axis=0)

# Print the entropy for each cluster
print("Entropy for each cluster:")
for cluster, entropy in cluster_entropies_sum.items():
    print(f"Cluster {cluster}: {entropy}")

Entropy for each cluster:
Cluster 0: 6.653853495735335
Cluster 1: 7.4264675741116655
Cluster 2: 7.187590761737598
Cluster 3: 7.636028116962557


In [ ]:
# Group by 'cluster' and count the occurrences of each drug in each cluster
cluster_counts = model1_df.groupby('labels')['Region'].value_counts().unstack(fill_value=0)

# Calculate the total count of drugs in each cluster
total_drugs_in_cluster = cluster_counts.sum(axis=1)

# Calculate the probability of each drug in each cluster
cluster_probabilities = cluster_counts.div(total_drugs_in_cluster, axis=0)

# Transpose the DataFrame to have drugs as rows and clusters as columns
cluster_probabilities_transposed = cluster_probabilities.T

# Calculate the entropy for each cluster
cluster_entropies = -(cluster_probabilities_transposed * np.log2(cluster_probabilities_transposed)).fillna(0)

# Sum the entropies for each cluster
cluster_entropies_sum = cluster_entropies.sum(axis=0)

# Print the entropy for each cluster
print("Entropy for each cluster:")
for cluster, entropy in cluster_entropies_sum.items():
    print(f"Cluster {cluster}: {entropy}")

Entropy for each cluster:
Cluster 0: 2.461677893126962
Cluster 1: 2.456127357197214
Cluster 2: 2.4862352450733702
Cluster 3: 2.45278527677163


In [ ]:
# Group by 'cluster' and count the occurrences of each drug in each cluster
cluster_counts = model1_df.groupby('labels')['disease'].value_counts().unstack(fill_value=0)

# Calculate the total count of drugs in each cluster
total_drugs_in_cluster = cluster_counts.sum(axis=1)

# Calculate the probability of each drug in each cluster
cluster_probabilities = cluster_counts.div(total_drugs_in_cluster, axis=0)

# Transpose the DataFrame to have drugs as rows and clusters as columns
cluster_probabilities_transposed = cluster_probabilities.T

# Calculate the entropy for each cluster
cluster_entropies = -(cluster_probabilities_transposed * np.log2(cluster_probabilities_transposed)).fillna(0)

# Sum the entropies for each cluster
cluster_entropies_sum = cluster_entropies.sum(axis=0)

# Print the entropy for each cluster
print("Entropy for each cluster:")
for cluster, entropy in cluster_entropies_sum.items():
    print(f"Cluster {cluster}: {entropy}")

Entropy for each cluster:
Cluster 0: 5.575212627649263
Cluster 1: 5.962066221476731
Cluster 2: 5.840946681689683
Cluster 3: 6.167231444498709


In [ ]:
# Accessing cluster centroids
centroids = pd.DataFrame(kp.cluster_centroids_,columns=['Tot_Clms', 'Tot_Drug_Cst', 'Tot_Day_Suply', 'Tot_Benes','Tot_30day_Fills'
                                                        ,'Gnrc_Name','Region', 'disease'])

# Add cluster titles (Cluster 1, Cluster 2, ...) to cluster_mean data frame with means and precision of 3 decimals.
centroids['labels'] = ['labels {}'.format(i) for i in centroids.index]
pd.set_option('display.precision', 3)

#Add 'Cluster number' column
pd.set_option('display.precision', 3)
print(centroids)



             Tot_Clms        Tot_Drug_Cst      Tot_Day_Suply  \
0   3.479158909891401  6.4209261815370855  7.352910719485349   
1  3.1482969457074734   9.546663961219238  6.727400031593992   
2   4.945683772309439   8.048307488212297   8.93239770553529   
3   2.824640360080091   5.235322656734042  5.868285445221641   

            Tot_Benes    Tot_30day_Fills                       Gnrc_Name  \
0   2.355995679313581  4.058920540894449            levothyroxine sodium   
1  2.0976578264457566  3.441635033913347  insulin glargine,hum.rec.anlog   
2  3.7384131006547086  5.592797303818603            atorvastatin calcium   
3  2.2019726855163615  3.016919654124315                     amoxicillin   

      Region            disease    labels  
0  Southeast             angina  labels 0  
1  Northeast  diabetes mellitus  labels 1  
2  Southeast             angina  labels 2  
3  Southeast               acne  labels 3  


###average cost

In [ ]:
# Calculate average drug cost for each cluster
average_drug_cost_per_cluster = model1_df.groupby('labels')['Tot_Drug_Cst'].mean()

# Calculate total drug cost for each cluster
total_drug_cost_per_cluster = model1_df.groupby('labels')['Tot_Drug_Cst'].sum()

# Print the results
print("Average Drug Cost for Each Cluster:")
print(average_drug_cost_per_cluster)

# Print the results
print("Total Drug Cost for Each Cluster:")
print(total_drug_cost_per_cluster)

Average Drug Cost for Each Cluster:
labels
0    14504.044569
1      798.121831
2    36249.395990
3      272.209422
Name: Tot_Drug_Cst, dtype: float64
Total Drug Cost for Each Cluster:
labels
0    23075934.91
1     2858872.40
2    57310295.06
3      857187.47
Name: Tot_Drug_Cst, dtype: float64


In [ ]:
# Group by cluster and aggregate unique drug names
unique_drug_names_per_cluster = model1_df.groupby('labels')['Gnrc_Name'].unique()

# Count the number of unique drugs in each cluster
unique_drugs_per_cluster = model1_df.groupby('labels')['Gnrc_Name'].nunique()

# Print the number of unique drugs and the unique drug names for each cluster
print("Number of unique drugs in each cluster and sorted unique drug names:")
for cluster, drugs_in_cluster in unique_drug_names_per_cluster.items():
    num_unique_drugs = unique_drugs_per_cluster.loc[cluster]
    print(f"Cluster {cluster}: {num_unique_drugs} unique drugs")
    sorted_drugs = sorted(drugs_in_cluster)
    print(", ".join(sorted_drugs))

Number of unique drugs in each cluster and sorted unique drug names:
Cluster 0: 224 unique drugs
albuterol sulfate, alendronate sodium, alirocumab, allopurinol, alprazolam, amantadine hcl, amiodarone hcl, amlodipine besylate, amlodipine besylate/valsartan, amoxicillin, anastrozole, apixaban, aripiprazole, atenolol, atorvastatin calcium, azelastine hcl, azithromycin, baclofen, benazepril hcl, benztropine mesylate, besifloxacin hcl, bimatoprost, bisoprolol fumarate, brimonidine tartrate, brimonidine tartrate/timolol, bromfenac sodium, budesonide/formoterol fumarate, bumetanide, buprenorphine hcl/naloxone hcl, bupropion hcl, buspirone hcl, calcium acetate, carbamazepine, carbidopa/levodopa, carvedilol, celecoxib, chlorhexidine gluconate, chlorthalidone, cilostazol, citalopram hydrobromide, clindamycin phosphate, clobetasol propionate, clonazepam, clopidogrel bisulfate, clozapine, cyclobenzaprine hcl, cyclosporine, diclofenac sodium, dicyclomine hcl, digoxin, diltiazem hcl, divalproex sodi

In [ ]:
## Convert NumPy arrays to sets for each cluster's unique drug names
unique_drug_sets_per_cluster = unique_drug_names_per_cluster.apply(set)

# Find the drugs present in all clusters
drug_in_all_clusters = set.intersection(*unique_drug_sets_per_cluster)

# Count of drugs present in all clusters
count_drug_in_all_clusters = len(drug_in_all_clusters)

# Print the drugs present in all clusters and their count
print("Drugs present in all clusters:")
print(", ".join(sorted(drug_in_all_clusters)))
print(f"Count: {count_drug_in_all_clusters}")

Drugs present in all clusters:
albuterol sulfate, amantadine hcl, aripiprazole, brimonidine tartrate, bumetanide, calcium acetate, carbamazepine, carbidopa/levodopa, celecoxib, clozapine, diclofenac sodium, diltiazem hcl, divalproex sodium, doxepin hcl, doxycycline hyclate, duloxetine hcl, esomeprazole magnesium, estradiol, fluticasone propionate, hydrocortisone, insulin lispro, ipratropium bromide, lamotrigine, latanoprost, levetiracetam, levothyroxine sodium, memantine hcl, metformin hcl, methotrexate sodium, mometasone furoate, morphine sulfate, olanzapine, oxycodone hcl, pantoprazole sodium, pramipexole di-hcl, pregabalin, propranolol hcl, quetiapine fumarate, risperidone, sucralfate, tolterodine tartrate, topiramate, venlafaxine hcl, ziprasidone hcl, zolpidem tartrate
Count: 45


In [ ]:
# Group by cluster and aggregate unique drug names
unique_disease_names_per_cluster = model1_df.groupby('labels')['disease'].unique()

# Count the number of unique drugs in each cluster
unique_disease_per_cluster = model1_df.groupby('labels')['disease'].nunique()

# Print the number of disease for each cluster
print("Number of disease in each cluster and sorted unique drug names:")
for cluster, drugs_in_cluster in unique_disease_names_per_cluster.items():
    num_unique_drugs = unique_disease_per_cluster.loc[cluster]
    print(f"Cluster {cluster}: {num_unique_drugs} unique disease")
    sorted_drugs = sorted(drugs_in_cluster)
    print(", ".join(sorted_drugs))

Number of disease in each cluster and sorted unique drug names:
Cluster 0: 167 unique disease
abdominal distension, abnormal uterine bleeding, abortion, acetaminophen overdose, acne, acromegaly, actinomycosis, acute coronary syndrome, acute lymphocytic leukemia, acute myeloid leukemia, acute otitis externa, addison's disease, adhd (attention deficit hyperactivity disorder), adjunct to antibiotic therapy, aggressive behavior, agitated state, agitation, aids related wasting, alcohol dependence, alcohol withdrawal, alkylating agent cystitis, allergic reactions, allergic rhinitis, allergic urticaria, alopecia, alport syndrome, alzheimer's disease, amenorrhea, amyloidosis, anal itching, androgenetic alopecia, anemia, megaloblastic, anesthesia, anesthetic adjunct, angina, angina pectoris prophylaxis, ankylosing spondylitis, anticholinesterase poisoning, antiphospholipid syndrome, anxiety, anxiety and stress, arrhythmia, ascites, aseptic necrosis, asthma, asthma, acute, asystole, atherosclero

In [ ]:
## Convert NumPy arrays to sets for each cluster's unique drug names
unique_drug_sets_per_cluster = unique_drug_names_per_cluster.apply(set)

# Find the drugs present in all clusters
drug_in_all_clusters = set.intersection(*unique_drug_sets_per_cluster)

# Count of drugs present in all clusters
count_drug_in_all_clusters = len(drug_in_all_clusters)

# Print the drugs present in all clusters and their count
print("Drugs present in all clusters:")
print(", ".join(sorted(drug_in_all_clusters)))
print(f"Count: {count_drug_in_all_clusters}")

Drugs present in all clusters:
acne, acute coronary syndrome, adhd (attention deficit hyperactivity disorder), agitated state, agitation, alcohol dependence, alzheimer's disease, angina, ankylosing spondylitis, anxiety, anxiety and stress, arrhythmia, asthma, asthma, acute, atopic dermatitis, atrial fibrillation, atrophic urethritis, autism, benign prostatic hyperplasia, bipolar disorder, borderline personality disorder, breast cancer, cardiovascular risk reduction, chronic pain, conjunctivitis, allergic, conjunctivitis, bacterial, dercum's disease, diabetes mellitus, diabetes, type 1, diabetes, type 2, ectopic pregnancy, edema, generalized anxiety disorder, gerd, glaucoma, open angle, hashimoto's disease (chronic lymphocytic thyroiditis), high blood pressure (hypertension), hyperphosphatemia, insomnia, migraine, mitral valve prolapse, muscle pain, occipital neuralgia, reflex sympathetic dystrophy syndrome, restless legs syndrome, rheumatoid arthritis, rosacea, seizures (convulsions), 

# Model 2 - K prototype

In [ ]:
model2_df = df_disease

In [ ]:
df_disease['Prscrbr_Type'].nunique()

66

In [ ]:
df_disease.columns

Index(['Prscrbr_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name',
       'Prscrbr_City', 'Prscrbr_State_Abrvtn', 'Prscrbr_State_FIPS',
       'Prscrbr_Type', 'Prscrbr_Type_Src', 'Brnd_Name', 'Gnrc_Name',
       'Tot_Clms', 'Tot_30day_Fills', 'Tot_Day_Suply', 'Tot_Drug_Cst',
       'Tot_Benes', 'GE65_Sprsn_Flag', 'GE65_Tot_Clms', 'GE65_Tot_30day_Fills',
       'GE65_Tot_Drug_Cst', 'GE65_Tot_Day_Suply', 'GE65_Bene_Sprsn_Flag',
       'GE65_Tot_Benes', 'Region', 'disease', 'labels'],
      dtype='object')

In [ ]:
cluster_model2_df = df_disease[['Gnrc_Name','Prscrbr_Type','Tot_Clms', 'Tot_Drug_Cst', 'Tot_Day_Suply',
                                'Tot_Benes','Tot_30day_Fills','Region', 'disease']]

In [ ]:
# Define the columns to apply log transformation
columns_to_transform = ['Tot_Clms', 'Tot_Drug_Cst', 'Tot_Day_Suply', 'Tot_Benes', 'Tot_30day_Fills']

# Apply log transformation to the selected columns
for column in columns_to_transform:
    cluster_model2_df[column] = np.log(cluster_model2_df[column] + 1)  # Adding 1 to handle zero values

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to the transformed columns
cluster_model2_df[columns_to_transform] = scaler.fit_transform(cluster_model2_df[columns_to_transform])

cluster_model2_df.head()

,Gnrc_Name,Prscrbr_Type,Tot_Clms,Tot_Drug_Cst,Tot_Day_Suply,Tot_Benes,Tot_30day_Fills,Region,disease
0,potassium chloride,Interventional Cardiology,0.405787,0.550000,0.674624,0.342799,0.477102,Southeast,hypokalemia
1,fluconazole,Family Practice,0.040869,0.173791,0.088099,0.000000,0.037841,Midwest,blastomycosis
2,acyclovir,Ophthalmology,0.285378,0.431907,0.590038,0.232090,0.352634,Southwest,varicella-zoster
3,fluticasone/vilanterol,Internal Medicine,0.111015,0.561067,0.440340,0.000000,0.117545,Midwest,atopic dermatitis
4,enalapril maleate,Family Practice,0.028233,0.295435,0.440340,0.000000,0.117545,Southeast,polycystic kidney disease


In [ ]:
# Convert dataframe to matrix
df_matrix = cluster_model2_df.to_numpy()
df_matrix

array([['potassium chloride', 'Interventional Cardiology',
        0.40578718651936857, ..., 0.4771016442505538, 'Southeast',
        'hypokalemia'],
       ['fluconazole', 'Family Practice', 0.04086923094620837, ...,
        0.037840957454843505, 'Midwest', 'blastomycosis'],
       ['acyclovir', 'Ophthalmology', 0.2853776049878655, ...,
        0.35263425021742945, 'Southwest', 'varicella-zoster'],
       ...,
       ['famotidine', 'Family Practice', 0.18527692618177843, ...,
        0.20835859021665948, 'Midwest', 'gerd'],
       ['haloperidol lactate', 'Psychiatry', 0.028233035179690835, ...,
        0.026141061584005587, 'Midwest', 'aggressive behavior'],
       ['nebivolol hcl', 'Cardiology', 0.21111578094542155, ...,
        0.3378787540565048, 'Southwest', 'mitral valve prolapse']],
      dtype=object)

In [ ]:
# Define the indices of the numerical and categorical features
cat_cols = [0,1,7,8]

# Perform k-prototype clustering
kp = KPrototypes(n_clusters=4, init='Cao', n_init=5, verbose=1)
kp.fit_predict(df_matrix, categorical=cat_cols)

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 2470, ncost: 2872.6377072754735
Run: 1, iteration: 2/100, moves: 783, ncost: 2866.782670039677
Run: 1, iteration: 3/100, moves: 329, ncost: 2865.6577564637782
Run: 1, iteration: 4/100, moves: 147, ncost: 2865.4097314435726
Run: 1, iteration: 5/100, moves: 94, ncost: 2865.3078271849017
Run: 1, iteration: 6/100, moves: 55, ncost: 2865.253156534887
Run: 1, iteration: 7/100, moves: 42, ncost: 2865.2212338528934
Run: 1, iteration: 8/100, moves: 27, ncost: 2865.2071684554694
Run: 1, iteration: 9/100, moves: 10, ncost: 2865.203506000767
Run: 1, iteration: 10/100, moves: 5, ncost: 2865.202658705214
Run: 1, iteration: 11/100, moves: 0, ncost: 2865.202658705214
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/100, moves: 2377, ncost: 2824.2511123151085
Run: 2

array([0, 2, 0, ..., 3, 2, 1], dtype=uint16)

In [ ]:
#new column for cluster labels associated with each subject
model2_df['labels'] = kp.labels_

In [ ]:
model2_df.head()

,Prscrbr_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_City,Prscrbr_State_Abrvtn,Prscrbr_State_FIPS,Prscrbr_Type,Prscrbr_Type_Src,Brnd_Name,Gnrc_Name,...,GE65_Sprsn_Flag,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Bene_Sprsn_Flag,GE65_Tot_Benes,Region,disease,labels
0,1003007980,Bernal,Juan,Birmingham,AL,1,Interventional Cardiology,S,Potassium Chloride,potassium chloride,...,NaN,75.0,143.0,7086.79,4290.0,#,NaN,Southeast,hypokalemia,0
1,1003073735,Fair,Keri,Chicago,IL,17,Family Practice,S,Fluconazole,fluconazole,...,#,NaN,NaN,NaN,NaN,*,NaN,Midwest,blastomycosis,2
2,1003075094,Di Pascuale,Mario,El Paso,TX,48,Ophthalmology,S,Acyclovir,acyclovir,...,NaN,56.0,95.0,1708.94,2740.0,NaN,19.0,Southwest,varicella-zoster,0
3,1003078007,Polsak,Micholee,Kansas City,KS,20,Internal Medicine,S,Breo Ellipta,fluticasone/vilanterol,...,NaN,21.0,23.0,9058.23,690.0,*,NaN,Midwest,atopic dermatitis,1
4,1003009119,Mohler,Troy,Purcellville,VA,51,Family Practice,S,Enalapril Maleate,enalapril maleate,...,NaN,13.0,23.0,292.68,690.0,*,NaN,Southeast,polycystic kidney disease,2


In [ ]:
for i in range(0,4):
   print ("Cluster",i,":", model2_df.groupby('labels').count().iloc[i]['Gnrc_Name'])

Cluster 0 : 2070
Cluster 1 : 2504
Cluster 2 : 2474
Cluster 3 : 2855


In [ ]:
# Group by 'cluster' and count the occurrences of each drug in each cluster
cluster_counts = model2_df.groupby('labels')['Gnrc_Name'].value_counts().unstack(fill_value=0)

# Calculate the total count of drugs in each cluster
total_drugs_in_cluster = cluster_counts.sum(axis=1)

# Calculate the probability of each drug in each cluster
cluster_probabilities = cluster_counts.div(total_drugs_in_cluster, axis=0)

# Transpose the DataFrame to have drugs as rows and clusters as columns
cluster_probabilities_transposed = cluster_probabilities.T

# Calculate the entropy for each cluster
cluster_entropies = -(cluster_probabilities_transposed * np.log2(cluster_probabilities_transposed)).fillna(0)

# Sum the entropies for each cluster
cluster_entropies_sum = cluster_entropies.sum(axis=0)

# Print the entropy for each cluster
print("Entropy for each cluster:")
for cluster, entropy in cluster_entropies_sum.items():
    print(f"Cluster {cluster}: {entropy}")

Entropy for each cluster:
Cluster 0: 6.885057262166768
Cluster 1: 8.026780766372536
Cluster 2: 7.801762438165581
Cluster 3: 8.088985612344285


In [ ]:
# Group by 'cluster' and count the occurrences of each drug in each cluster
cluster_counts = model2_df.groupby('labels')['Region'].value_counts().unstack(fill_value=0)

# Calculate the total count of drugs in each cluster
total_drugs_in_cluster = cluster_counts.sum(axis=1)

# Calculate the probability of each drug in each cluster
cluster_probabilities = cluster_counts.div(total_drugs_in_cluster, axis=0)

# Transpose the DataFrame to have drugs as rows and clusters as columns
cluster_probabilities_transposed = cluster_probabilities.T

# Calculate the entropy for each cluster
cluster_entropies = -(cluster_probabilities_transposed * np.log2(cluster_probabilities_transposed)).fillna(0)

# Sum the entropies for each cluster
cluster_entropies_sum = cluster_entropies.sum(axis=0)

# Print the entropy for each cluster
print("Entropy for each cluster:")
for cluster, entropy in cluster_entropies_sum.items():
    print(f"Cluster {cluster}: {entropy}")

Entropy for each cluster:
Cluster 0: 2.384935597213398
Cluster 1: 2.035647060091033
Cluster 2: 2.0777517200378774
Cluster 3: 2.099147522601393


In [ ]:
# Group by 'cluster' and count the occurrences of each drug in each cluster
cluster_counts = model2_df.groupby('labels')['disease'].value_counts().unstack(fill_value=0)

# Calculate the total count of drugs in each cluster
total_drugs_in_cluster = cluster_counts.sum(axis=1)

# Calculate the probability of each drug in each cluster
cluster_probabilities = cluster_counts.div(total_drugs_in_cluster, axis=0)

# Transpose the DataFrame to have drugs as rows and clusters as columns
cluster_probabilities_transposed = cluster_probabilities.T

# Calculate the entropy for each cluster
cluster_entropies = -(cluster_probabilities_transposed * np.log2(cluster_probabilities_transposed)).fillna(0)

# Sum the entropies for each cluster
cluster_entropies_sum = cluster_entropies.sum(axis=0)

# Print the entropy for each cluster
print("Entropy for each cluster:")
for cluster, entropy in cluster_entropies_sum.items():
    print(f"Cluster {cluster}: {entropy}")

Entropy for each cluster:
Cluster 0: 5.693358536679801
Cluster 1: 6.388412514291255
Cluster 2: 6.276127256924211
Cluster 3: 6.462452500311249


In [ ]:
cluster_model2_df.columns

Index(['Gnrc_Name', 'Prscrbr_Type', 'Tot_Clms', 'Tot_Drug_Cst',
       'Tot_Day_Suply', 'Tot_Benes', 'Tot_30day_Fills', 'Region', 'disease'],
      dtype='object')

In [ ]:
# Accessing cluster centroids
centroids = pd.DataFrame(kp.cluster_centroids_,columns=['Gnrc_Name', 'Prscrbr_Type', 'Tot_Clms', 'Tot_Drug_Cst',
       'Tot_Day_Suply', 'Tot_Benes', 'Tot_30day_Fills', 'Region', 'disease'])

# Add cluster titles (Cluster 1, Cluster 2, ...) to cluster_mean data frame with means and precision of 3 decimals.
centroids['labels'] = ['labels {}'.format(i) for i in centroids.index]
pd.set_option('display.precision', 3)

#Add 'Cluster number' column
pd.set_option('display.precision', 3)
print(centroids)



             Gnrc_Name        Prscrbr_Type             Tot_Clms  \
0  0.41574761693540707  0.4614391280947601   0.6714123592973457   
1   0.1651690438630801  0.4058977777332338   0.5030535378745029   
2  0.07200377479504151  0.2947105161090314   0.3747574267107892   
3  0.10528586518850591  0.3796027856245692  0.46106801599616964   

           Tot_Drug_Cst        Tot_Day_Suply                       Tot_Benes  \
0    0.3666226421419997   0.4897901388842496            atorvastatin calcium   
1   0.07501836578209795    0.231979363825967                   trazodone hcl   
2   0.06276993019296986  0.10273468902923051                     amoxicillin   
3  0.027313173769839345  0.16915406659739157  insulin glargine,hum.rec.anlog   

      Tot_30day_Fills     Region  disease    labels  
0     Family Practice  Southeast   angina  labels 0  
1   Internal Medicine  Northeast  anxiety  labels 1  
2  Nurse Practitioner  Southeast     acne  labels 2  
3     Family Practice       West  anxiety  labe

In [ ]:
# Calculate average drug cost for each cluster
average_drug_cost_per_cluster = model2_df.groupby('labels')['Tot_Drug_Cst'].mean()

# Calculate total drug cost for each cluster
total_drug_cost_per_cluster = model2_df.groupby('labels')['Tot_Drug_Cst'].sum()

# Print the results
print("Average Drug Cost for Each Cluster:")
print(average_drug_cost_per_cluster)

# Print the results
print("Total Drug Cost for Each Cluster:")
print(total_drug_cost_per_cluster)

Average Drug Cost for Each Cluster:
labels
0    16648.904
1    11398.186
2     1764.946
3     5860.429
Name: Tot_Drug_Cst, dtype: float64
Total Drug Cost for Each Cluster:
labels
0    3.446e+07
1    2.854e+07
2    4.366e+06
3    1.673e+07
Name: Tot_Drug_Cst, dtype: float64


In [ ]:
# Group by cluster and aggregate unique drug names
unique_drug_names_per_cluster = model2_df.groupby('labels')['Gnrc_Name'].unique()

# Count the number of unique drugs in each cluster
unique_drugs_per_cluster = model2_df.groupby('labels')['Gnrc_Name'].nunique()

# Print the number of unique drugs and the unique drug names for each cluster
print("Number of unique drugs in each cluster and sorted unique drug names:")
for cluster, drugs_in_cluster in unique_drug_names_per_cluster.items():
    num_unique_drugs = unique_drugs_per_cluster.loc[cluster]
    print(f"Cluster {cluster}: {num_unique_drugs} unique drugs")
    sorted_drugs = sorted(drugs_in_cluster)
    print(", ".join(sorted_drugs))

Number of unique drugs in each cluster and sorted unique drug names:
Cluster 0: 265 unique drugs
acetaminophen with codeine, acyclovir, albuterol sulfate, alendronate sodium, alirocumab, allopurinol, alprazolam, amantadine hcl, amiodarone hcl, amitriptyline hcl, amlodipine besylate, amlodipine besylate/valsartan, ammonium lactate, amoxicillin, amoxicillin/potassium clav, anastrozole, apixaban, aripiprazole, atenolol, atorvastatin calcium, azelastine hcl, azithromycin, bacitracin, baclofen, benazepril hcl, benztropine mesylate, besifloxacin hcl, bictegrav/emtricit/tenofov ala, bimatoprost, bisoprolol fumarate, brimonidine tartrate, brimonidine tartrate/timolol, brinzolamide/brimonidine tart, bromfenac sodium, budesonide/formoterol fumarate, budesonide/glycopyr/formoterol, bumetanide, buprenorphine hcl/naloxone hcl, bupropion hcl, buspirone hcl, calcium acetate, carbamazepine, carbidopa/levodopa, cariprazine hcl, carvedilol, cefadroxil, cefdinir, celecoxib, cephalexin, chlorhexidine gluc

In [ ]:
# Group by cluster and aggregate unique drug names
unique_disease_names_per_cluster = model2_df.groupby('labels')['disease'].unique()

# Count the number of unique drugs in each cluster
unique_disease_per_cluster = model2_df.groupby('labels')['disease'].nunique()

# Print the number of disease for each cluster
print("Number of disease in each cluster and sorted unique drug names:")
for cluster, drugs_in_cluster in unique_disease_names_per_cluster.items():
    num_unique_drugs = unique_disease_per_cluster.loc[cluster]
    print(f"Cluster {cluster}: {num_unique_drugs} unique disease")
    sorted_drugs = sorted(drugs_in_cluster)
    print(", ".join(sorted_drugs))

Number of disease in each cluster and sorted unique drug names:
Cluster 0: 130 unique disease
acne, actinic keratosis, actinomycosis, acute coronary syndrome, acute lymphocytic leukemia, acute otitis externa, addison's disease, adhd (attention deficit hyperactivity disorder), aggressive behavior, agitated state, agitation, aids/hiv, alcohol dependence, alcohol withdrawal, alkylating agent cystitis, allergic reactions, allergic rhinitis, allergic urticaria, alport syndrome, alzheimer's disease, androgenetic alopecia, anemia, angina, angina pectoris prophylaxis, ankylosing spondylitis, antiphospholipid syndrome, anxiety, anxiety and stress, arrhythmia, aseptic necrosis, asthma, asthma, acute, asthma, maintenance, atherosclerosis, atopic dermatitis, atrial fibrillation, atrophic urethritis, autism, back pain, bacteremia, bacterial endocarditis prevention (bacterial endocarditis prophylaxis), bacterial skin infection, benign prostatic hyperplasia, bipolar disorder, borderline personality d

In [ ]:
## Convert NumPy arrays to sets for each cluster's unique drug names
unique_drug_sets_per_cluster = unique_drug_names_per_cluster.apply(set)

# Find the drugs present in all clusters
drug_in_all_clusters = set.intersection(*unique_drug_sets_per_cluster)

# Count of drugs present in all clusters
count_drug_in_all_clusters = len(drug_in_all_clusters)

# Print the drugs present in all clusters and their count
print("Drugs present in all clusters:")
print(", ".join(sorted(drug_in_all_clusters)))
print(f"Count: {count_drug_in_all_clusters}")

Drugs present in all clusters:
acetaminophen with codeine, acyclovir, albuterol sulfate, alendronate sodium, alirocumab, allopurinol, alprazolam, amantadine hcl, amiodarone hcl, amitriptyline hcl, amlodipine besylate, amoxicillin/potassium clav, anastrozole, apixaban, aripiprazole, atenolol, atorvastatin calcium, azelastine hcl, azithromycin, baclofen, benazepril hcl, benztropine mesylate, bisoprolol fumarate, brimonidine tartrate, brinzolamide/brimonidine tart, budesonide/formoterol fumarate, bumetanide, buprenorphine hcl/naloxone hcl, bupropion hcl, buspirone hcl, carbamazepine, carbidopa/levodopa, carvedilol, cefdinir, celecoxib, cephalexin, chlorhexidine gluconate, chlorthalidone, cilostazol, ciprofloxacin hcl, citalopram hydrobromide, clobetasol propionate, clonazepam, clonidine hcl, clopidogrel bisulfate, cyclobenzaprine hcl, cyclosporine, dexamethasone, diazepam, diclofenac sodium, dicyclomine hcl, digoxin, diltiazem hcl, divalproex sodium, donepezil hcl, dorzolamide hcl, doxazo

# Model 3 - K modes

In [ ]:
model3_df = df_disease

In [ ]:
df_disease['Prscrbr_Type'].nunique()

66

In [ ]:
df_disease.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9903 entries, 0 to 9953
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Prscrbr_NPI            9903 non-null   int64  
 1   Prscrbr_Last_Org_Name  9903 non-null   object 
 2   Prscrbr_First_Name     9903 non-null   object 
 3   Prscrbr_City           9903 non-null   object 
 4   Prscrbr_State_Abrvtn   9903 non-null   object 
 5   Prscrbr_State_FIPS     9903 non-null   int64  
 6   Prscrbr_Type           9903 non-null   object 
 7   Prscrbr_Type_Src       9903 non-null   object 
 8   Brnd_Name              9903 non-null   object 
 9   Gnrc_Name              9903 non-null   object 
 10  Tot_Clms               9903 non-null   int64  
 11  Tot_30day_Fills        9903 non-null   float64
 12  Tot_Day_Suply          9903 non-null   int64  
 13  Tot_Drug_Cst           9903 non-null   float64
 14  Tot_Benes              9903 non-null   int64  
 15  GE65_Sprs

In [ ]:
cluster_model3_df = df_disease[['Brnd_Name','Gnrc_Name','Prscrbr_Type','Region', 'disease']]

In [ ]:
# Convert dataframe to matrix
df_matrix = cluster_model3_df.to_numpy()
df_matrix

array([['Potassium Chloride', 'potassium chloride',
        'Interventional Cardiology', 'Southeast', 'hypokalemia'],
       ['Fluconazole', 'fluconazole', 'Family Practice', 'Midwest',
        'blastomycosis'],
       ['Acyclovir', 'acyclovir', 'Ophthalmology', 'Southwest',
        'varicella-zoster'],
       ...,
       ['Famotidine', 'famotidine', 'Family Practice', 'Midwest', 'gerd'],
       ['Haloperidol Lactate', 'haloperidol lactate', 'Psychiatry',
        'Midwest', 'aggressive behavior'],
       ['Bystolic', 'nebivolol hcl', 'Cardiology', 'Southwest',
        'mitral valve prolapse']], dtype=object)

In [ ]:
from kmodes.kmodes import KModes

# Define your k-prototype clustering model
kp = KModes(n_clusters=4, init='Cao', n_init=5, verbose=1)

# Fit the model and perform clustering on your data
cluster_labels = kp.fit_predict(df_matrix)

# Print cluster centroids
print("Cluster Centroids:")
print(kp.cluster_centroids_)

# Print cluster labels for each data point
print("Cluster Labels:")
print(cluster_labels)

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 0, cost: 37955.0
Cluster Centroids:
[['Amlodipine Besylate' 'amlodipine besylate' 'Family Practice'
  'Southeast' 'angina']
 ['Trazodone Hcl' 'trazodone hcl' 'Internal Medicine' 'Northeast'
  'anxiety']
 ['Omeprazole' 'omeprazole' 'Family Practice' 'Northeast' 'gerd']
 ['Duloxetine Hcl' 'duloxetine hcl' 'Family Practice' 'West' 'anxiety']]
Cluster Labels:
[0 0 0 ... 2 0 0]


In [ ]:
#new column for cluster labels associated with each subject
model3_df['labels'] = kp.labels_

In [ ]:
model3_df.head()

,Prscrbr_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_City,Prscrbr_State_Abrvtn,Prscrbr_State_FIPS,Prscrbr_Type,Prscrbr_Type_Src,Brnd_Name,Gnrc_Name,...,GE65_Sprsn_Flag,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Bene_Sprsn_Flag,GE65_Tot_Benes,Region,disease,labels
0,1003007980,Bernal,Juan,Birmingham,AL,1,Interventional Cardiology,S,Potassium Chloride,potassium chloride,...,NaN,75.0,143.0,7086.79,4290.0,#,NaN,Southeast,hypokalemia,0
1,1003073735,Fair,Keri,Chicago,IL,17,Family Practice,S,Fluconazole,fluconazole,...,#,NaN,NaN,NaN,NaN,*,NaN,Midwest,blastomycosis,0
2,1003075094,Di Pascuale,Mario,El Paso,TX,48,Ophthalmology,S,Acyclovir,acyclovir,...,NaN,56.0,95.0,1708.94,2740.0,NaN,19.0,Southwest,varicella-zoster,0
3,1003078007,Polsak,Micholee,Kansas City,KS,20,Internal Medicine,S,Breo Ellipta,fluticasone/vilanterol,...,NaN,21.0,23.0,9058.23,690.0,*,NaN,Midwest,atopic dermatitis,1
4,1003009119,Mohler,Troy,Purcellville,VA,51,Family Practice,S,Enalapril Maleate,enalapril maleate,...,NaN,13.0,23.0,292.68,690.0,*,NaN,Southeast,polycystic kidney disease,0


from matplotlib import pyplot as plt
_df_0['Prscrbr_NPI'].plot(kind='hist', bins=20, title='Prscrbr_NPI')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Prscrbr_State_FIPS'].plot(kind='hist', bins=20, title='Prscrbr_State_FIPS')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Tot_Clms'].plot(kind='hist', bins=20, title='Tot_Clms')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Tot_30day_Fills'].plot(kind='hist', bins=20, title='Tot_30day_Fills')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Prscrbr_Type'].value_counts()
    for x_label, grp in _df_19.groupby('Prscrbr_State_Abrvtn')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Prscrbr_State_Abrvtn')
_ = plt.ylabel('Prscrbr_Type')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['Prscrbr_Last_Org_Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='Prscrbr_NPI', y='Prscrbr_Last_Org_Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['Prscrbr_First_Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='Prscrbr_NPI', y='Prscrbr_First_Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_22['Prscrbr_City'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_22, x='Prscrbr_NPI', y='Prscrbr_City', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_23['Prscrbr_State_Abrvtn'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_23, x='Prscrbr_NPI', y='Prscrbr_State_Abrvtn', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
for i in range(0,4):
   print ("Cluster",i,":", model3_df.groupby('labels').count().iloc[i]['Gnrc_Name'])

Cluster 0 : 5185
Cluster 1 : 2666
Cluster 2 : 542
Cluster 3 : 1510


In [ ]:
# Group by 'cluster' and count the occurrences of each drug in each cluster
cluster_counts = model3_df.groupby('labels')['Gnrc_Name'].value_counts().unstack(fill_value=0)

# Calculate the total count of drugs in each cluster
total_drugs_in_cluster = cluster_counts.sum(axis=1)

# Calculate the probability of each drug in each cluster
cluster_probabilities = cluster_counts.div(total_drugs_in_cluster, axis=0)

# Transpose the DataFrame to have drugs as rows and clusters as columns
cluster_probabilities_transposed = cluster_probabilities.T

# Calculate the entropy for each cluster
cluster_entropies = -(cluster_probabilities_transposed * np.log2(cluster_probabilities_transposed)).fillna(0)

# Sum the entropies for each cluster
cluster_entropies_sum = cluster_entropies.sum(axis=0)

# Print the entropy for each cluster
print("Entropy for each cluster:")
for cluster, entropy in cluster_entropies_sum.items():
    print(f"Cluster {cluster}: {entropy}")

Entropy for each cluster:
Cluster 0: 8.034699987330804
Cluster 1: 7.885366136638314
Cluster 2: 5.367593574266142
Cluster 3: 7.593272518666195


In [ ]:
# Group by 'cluster' and count the occurrences of each drug in each cluster
cluster_counts = model3_df.groupby('labels')['Region'].value_counts().unstack(fill_value=0)

# Calculate the total count of drugs in each cluster
total_drugs_in_cluster = cluster_counts.sum(axis=1)

# Calculate the probability of each drug in each cluster
cluster_probabilities = cluster_counts.div(total_drugs_in_cluster, axis=0)

# Transpose the DataFrame to have drugs as rows and clusters as columns
cluster_probabilities_transposed = cluster_probabilities.T

# Calculate the entropy for each cluster
cluster_entropies = -(cluster_probabilities_transposed * np.log2(cluster_probabilities_transposed)).fillna(0)

# Sum the entropies for each cluster
cluster_entropies_sum = cluster_entropies.sum(axis=0)

# Print the entropy for each cluster
print("Entropy for each cluster:")
for cluster, entropy in cluster_entropies_sum.items():
    print(f"Cluster {cluster}: {entropy}")

Entropy for each cluster:
Cluster 0: 1.8934633272319903
Cluster 1: 1.7138520858290445
Cluster 2: 1.6898605527395765
Cluster 3: 0.3591676173793768


In [ ]:
# Group by 'cluster' and count the occurrences of each drug in each cluster
cluster_counts = model3_df.groupby('labels')['disease'].value_counts().unstack(fill_value=0)

# Calculate the total count of drugs in each cluster
total_drugs_in_cluster = cluster_counts.sum(axis=1)

# Calculate the probability of each drug in each cluster
cluster_probabilities = cluster_counts.div(total_drugs_in_cluster, axis=0)

# Transpose the DataFrame to have drugs as rows and clusters as columns
cluster_probabilities_transposed = cluster_probabilities.T

# Calculate the entropy for each cluster
cluster_entropies = -(cluster_probabilities_transposed * np.log2(cluster_probabilities_transposed)).fillna(0)

# Sum the entropies for each cluster
cluster_entropies_sum = cluster_entropies.sum(axis=0)

# Print the entropy for each cluster
print("Entropy for each cluster:")
for cluster, entropy in cluster_entropies_sum.items():
    print(f"Cluster {cluster}: {entropy}")

Entropy for each cluster:
Cluster 0: 6.3705907169097635
Cluster 1: 6.384113019798155
Cluster 2: 3.893708305380271
Cluster 3: 6.233927108907468


In [ ]:
cluster_model3_df.columns

Index(['Brnd_Name', 'Gnrc_Name', 'Prscrbr_Type', 'Region', 'disease'], dtype='object')

In [ ]:
# Accessing cluster centroids
centroids = pd.DataFrame(kp.cluster_centroids_,columns=['Brnd_Name', 'Gnrc_Name', 'Prscrbr_Type', 'Region', 'disease'])

# Add cluster titles (Cluster 1, Cluster 2, ...) to cluster_mean data frame with means and precision of 3 decimals.
centroids['labels'] = ['labels {}'.format(i) for i in centroids.index]
pd.set_option('display.precision', 3)

#Add 'Cluster number' column
pd.set_option('display.precision', 3)
print(centroids)



             Brnd_Name            Gnrc_Name       Prscrbr_Type     Region  \
0  Amlodipine Besylate  amlodipine besylate    Family Practice  Southeast   
1        Trazodone Hcl        trazodone hcl  Internal Medicine  Northeast   
2           Omeprazole           omeprazole    Family Practice  Northeast   
3       Duloxetine Hcl       duloxetine hcl    Family Practice       West   

   disease    labels  
0   angina  labels 0  
1  anxiety  labels 1  
2     gerd  labels 2  
3  anxiety  labels 3  


In [ ]:
# Calculate average drug cost for each cluster
average_drug_cost_per_cluster = model3_df.groupby('labels')['Tot_Drug_Cst'].mean()

# Calculate total drug cost for each cluster
total_drug_cost_per_cluster = model3_df.groupby('labels')['Tot_Drug_Cst'].sum()

# Print the results
print("Average Drug Cost for Each Cluster:")
print(average_drug_cost_per_cluster)

# Print the results
print("Total Drug Cost for Each Cluster:")
print(total_drug_cost_per_cluster)

Average Drug Cost for Each Cluster:
labels
0    9806.698
1    7515.383
2    2270.213
3    7939.134
Name: Tot_Drug_Cst, dtype: float64
Total Drug Cost for Each Cluster:
labels
0    5.085e+07
1    2.004e+07
2    1.230e+06
3    1.199e+07
Name: Tot_Drug_Cst, dtype: float64


In [ ]:
# Group by cluster and aggregate unique drug names
unique_drug_names_per_cluster = model3_df.groupby('labels')['Gnrc_Name'].unique()

# Count the number of unique drugs in each cluster
unique_drugs_per_cluster = model3_df.groupby('labels')['Gnrc_Name'].nunique()

# Print the number of unique drugs and the unique drug names for each cluster
print("Number of unique drugs in each cluster and sorted unique drug names:")
for cluster, drugs_in_cluster in unique_drug_names_per_cluster.items():
    num_unique_drugs = unique_drugs_per_cluster.loc[cluster]
    print(f"Cluster {cluster}: {num_unique_drugs} unique drugs")
    sorted_drugs = sorted(drugs_in_cluster)
    print(", ".join(sorted_drugs))

Number of unique drugs in each cluster and sorted unique drug names:
Cluster 0: 543 unique drugs
0.9 % sodium chloride, abacavir/dolutegravir/lamivudi, abaloparatide, abiraterone acetate, acalabrutinib, acarbose, acetaminophen with codeine, acetazolamide, acyclovir, adalimumab, albuterol sulfate, alendronate sodium, alfuzosin hcl, alirocumab, allopurinol, alprazolam, amantadine hcl, amiloride hcl, amiodarone hcl, amitriptyline hcl, amlodipine besylate, amlodipine besylate/benazepril, amlodipine besylate/valsartan, ammonium lactate, amoxicillin, amoxicillin/potassium clav, ampicillin trihydrate, anastrozole, apixaban, aripiprazole, aripiprazole lauroxil, atenolol, atenolol/chlorthalidone, atomoxetine hcl, atorvastatin calcium, atropine sulfate, azathioprine, azelastine hcl, azelastine/fluticasone, azithromycin, bacitracin, baclofen, balsalazide disodium, beclomethasone dipropionate, belimumab, bempedoic acid, benazepril hcl, benzonatate, benztropine mesylate, besifloxacin hcl, betametha

In [ ]:
# Group by cluster and aggregate unique drug names
unique_disease_names_per_cluster = model3_df.groupby('labels')['disease'].unique()

# Count the number of unique drugs in each cluster
unique_disease_per_cluster = model3_df.groupby('labels')['disease'].nunique()

# Print the number of disease for each cluster
print("Number of disease in each cluster and sorted unique drug names:")
for cluster, drugs_in_cluster in unique_disease_names_per_cluster.items():
    num_unique_drugs = unique_disease_per_cluster.loc[cluster]
    print(f"Cluster {cluster}: {num_unique_drugs} unique disease")
    sorted_drugs = sorted(drugs_in_cluster)
    print(", ".join(sorted_drugs))

Number of disease in each cluster and sorted unique drug names:
Cluster 0: 222 unique disease
abnormal uterine bleeding, abortion, acne, acromegaly, actinic keratosis, actinomycosis, acute coronary syndrome, acute lymphoblastic leukemia, acute lymphocytic leukemia, acute myeloid leukemia, acute otitis externa, addison's disease, adhd (attention deficit hyperactivity disorder), aggressive behavior, agitated state, agitation, aids related wasting, aids/hiv, alcohol dependence, alcohol withdrawal, alkylating agent cystitis, allergic asthma, allergic reactions, allergic rhinitis, allergic urticaria, alopecia, alport syndrome, alzheimer's disease, amenorrhea, amyloidosis, anal itching, androgenetic alopecia, anemia, anemia, megaloblastic, anesthesia, anesthetic adjunct, angina, angina pectoris prophylaxis, ankylosing spondylitis, anti nmda receptor encephalitis, anticholinesterase poisoning, antiphospholipid syndrome, anxiety, anxiety and stress, arrhythmia, ascites, aseptic necrosis, aspir